# Bharath Gunasekaran
Avalance Continual Learning on Omniglot dataset

In [6]:
!pip install git+https://github.com/ContinualAI/avalanche.git


  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-4e5ph2pl
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-4e5ph2pl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [7]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitOmniglot
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP, as_multitask
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive
from torchvision import transforms

In [8]:
height = 28

scenario = SplitOmniglot(n_experiences=4, 
                          seed=1, 
                          train_transform=transforms.Compose([
                              transforms.Resize(height),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.9221,), (0.2681,))
                          ]), 
                          eval_transform=transforms.Compose([
                              transforms.Resize(height),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.9221,), (0.2681,))
                          ]))

print(scenario.n_classes)
model = SimpleMLP(num_classes=scenario.n_classes, input_size=height*height)


# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)
# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", len(experience.classes_in_this_experience))
    print('Task {} batch {} -> train'.format(experience.task_label, experience.current_experience))
    print('This batch contains', len(experience.dataset), 'patterns')   

    # train returns a dictionary which contains all the metric values  
    res = cl_strategy.train(experience)
    print('Training completed')
    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Files already downloaded and verified
Files already downloaded and verified
964
Starting experiment...
Start of experience:  0
Current Classes:  241
Task 0 batch 0 -> train
This batch contains 4820 patterns
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:84: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


100%|██████████| 10/10 [00:03<00:00,  3.21it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55684.7773
	DiskUsage_MB/train_phase/train_stream/Task000 = 55684.7773
	Loss_Epoch/train_phase/train_stream/Task000 = 6.9018
	Loss_MB/train_phase/train_stream/Task000 = 6.9112
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0335
	Time_Epoch/train_phase/train_stream/Task000 = 3.1090
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0017
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 32/32 [00:02<00:00, 14.61it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 110.9808
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 55685.3623
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 6.8665
	To